<a href="https://colab.research.google.com/github/Leandro2402-bit/Senales-Sistemas/blob/main/Parcial_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Ejercicio 1: En este problema se busca adecuar la señal x(t) = 20sen(7t-π/2) - 3cos(5t) + 2cos(10t), a un microprocesador de 5 bits, con un rango de entrada en [-3.3V , 5V]. Diseñando el sistema de acondicionamiento y digitalizacion de dicha señal x(t). Se presentan 2 tipos de soluciones.

In [ ]:
# Propuesta 1

import numpy as np
import matplotlib.pyplot as plt

# Paso 1: Definir el tiempo - al menos dos períodos de la señal más lenta (7t)
f_base = 7  # frecuencia más baja (7 rad/s → T = 2π/7)
T = 2 * np.pi / f_base
t = np.linspace(0, 2 * T * 2, 1000)  # dos períodos

# Paso 2: Definir la señal original x(t)
x = 20 * np.sin(7 * t - np.pi / 2) - 3 * np.cos(5 * t) + 2 * np.cos(10 * t)

# Paso 3: Acondicionamiento con "Cero y pendiente"
m = 0.166  # pendiente calculada
b = 0.85   # desplazamiento calculado
y = m * x + b  # señal acondicionada

# Paso 4: Graficar señal original y acondicionada
plt.figure(figsize=(12, 5))

plt.subplot(2, 1, 1)
plt.plot(t, x, label='Señal original x(t)', color='blue')
plt.title('Señal original x(t)')
plt.grid(True)
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(t, y, label='Señal acondicionada y(t)', color='orange')
plt.title('Señal acondicionada y(t) (0.166·x + 0.85)')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Paso 5: Cuantización - 5 bits → 2^5 = 32 niveles
niveles = 2 ** 5
Vmin = -3.3
Vmax = 5
paso = (Vmax - Vmin) / (niveles - 1)  # paso de cuantización

# Cuantización: redondeamos la señal acondicionada al nivel más cercano
y_cuantizada = np.round((y - Vmin) / paso) * paso + Vmin

# Paso 6: Graficar señal cuantizada
plt.figure(figsize=(10, 4))
plt.plot(t, y, label='Señal acondicionada', alpha=0.5)
plt.step(t, y_cuantizada, label='Señal cuantizada (5 bits)', color='red', where='mid')
plt.title('Señal acondicionada y su cuantización (5 bits)')
plt.grid(True)
plt.legend()
plt.show()


In [ ]:
# Propuesta 2
import numpy as np
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist  # Para calcular distancias entre puntos (útil en la cuantización)

# --- PARÁMETROS DE CUANTIZACIÓN ---
y_min, y_max = -3.3, 5        # Rango de valores de amplitud permitidos del microprocesador
nbits = 5                     # Número de bits del microprocesador
n_levels = 2**nbits           # Número de niveles posibles (2^n)
ve = np.linspace(y_min, y_max, n_levels)  # Vector con los niveles de cuantización uniformemente espaciados

# --- FUNCIÓN DE AJUSTE DE CERO Y PENDIENTE ---
def cero_pen(x, ymin=-25, ymax=20):
    """Aplica una transformación lineal a la señal x para que encaje entre ymin y ymax"""
    m = (ymax - ymin) / (x.max() - x.min())  # Cálculo de la pendiente
    c = ymin - m * x.min()                   # Cálculo de la ordenada al origen
    return m * x + c                         # Retorna la señal ajustada

# --- GENERACIÓN DE SEÑAL ---
Fo = 1.59                   # Frecuencia fundamental de la señal (Hz)
Fs = 10 * Fo                # Frecuencia de muestreo (mínimo 10 veces la fundamental para buena resolución)
Ts = 1 / Fs                 # Periodo de muestreo
To = 1 / Fo                 # Periodo de la señal original
tv = np.arange(0, 5 * To, Ts)  # Vector de tiempo desde 0 hasta 5 periodos
# Generación de la señal compuesta por varias frecuencias
y = -20*np.cos(7*tv) - 3*np.cos(5*tv) + 2*np.cos(10*tv)

# Ajuste de pendiente y nivel para que esté dentro del rango de cuantización
ysp = cero_pen(y, y_min, y_max)

# --- CUANTIZACIÓN ---
# Crea una matriz de distancias entre cada valor de ysp y cada nivel de cuantización
d = cdist(ve.reshape(-1,1), ysp.reshape(-1,1))  # reshape(-1,1) convierte los vectores en columnas
indices = np.argmin(d, axis=0)  # Encuentra el índice del nivel más cercano para cada valor
yq = ve[indices]                # Señal cuantizada (asigna cada valor al nivel más cercano)

# --- GRAFICACIÓN ---
plt.figure(figsize=(12, 6))  # Crea una figura con tamaño definido (ancho x alto en pulgadas)

# Señal continua ajustada (después del ajuste de rango)
plt.plot(tv, ysp, color='#1f77b4', linewidth=2, label='Señal continua ajustada')

# Señal discreta (muestreada) con stem plot
plt.stem(tv, ysp, linefmt='--', markerfmt='go', basefmt=" ", label='Señal muestreada (discreta)')

# Líneas horizontales que muestran los niveles de cuantización
for v in ve:
    plt.axhline(y=v, color='gray', linestyle=':', linewidth=0.5, alpha=0.4)  # Línea horizontal tenue

# Señal cuantizada (línea por tramos con puntos)
plt.plot(tv, yq, 'o-', color='#ff7f0e', linewidth=2, markersize=4, label='Señal cuantizada')

# Etiquetas de los ejes
plt.xlabel('Tiempo [s]')
plt.ylabel('Amplitud')

# Título del gráfico
plt.title('Cuantización Uniforme con Visualización Mejorada')

# Cuadrícula suave para facilitar la lectura
plt.grid(True, which='both', linestyle='--', alpha=0.5)

# Muestra la leyenda
plt.legend()

# Ajusta los márgenes para que nada se corte
plt.tight_layout()

# Muestra la figura
plt.show()

Ejercicio 2: En este punto se busca probar o identificar si la señal  x(t) = 3cos(1000πt) + 5sen(2000πt) + 10cos(1100πt) que tiene una Fs (frecuencia de muestreo) de 5KHz es apropiado para discretizar dicha señal. Ademas se debera implementar un conversor A/D adecuado en caso de que no funcione.

In [ ]:
# Ejercicio 2: Simulación de muestreo de señales y el teorema de Nyquist

import numpy as np  # Importa la librería NumPy para cálculos numéricos
import matplotlib.pyplot as plt  # Importa matplotlib para visualización

# ========== Configuración de la señal continua original ==========
# Crea un array de tiempo continuo desde 0 hasta 0.005 segundos (5ms) con 1000 puntos
t_cont = np.linspace(0, 0.005, 1000)

# Definición de la señal compuesta por tres componentes sinusoidales:
# 1. 3*cos(1000πt) → Frecuencia = 500 Hz (1000π/(2π))
# 2. 5*sin(2000πt) → Frecuencia = 1000 Hz
# 3. 10*cos(11000πt) → Frecuencia = 5500 Hz
x_cont = 3 * np.cos(1000 * np.pi * t_cont) + \
         5 * np.sin(2000 * np.pi * t_cont) + \
         10 * np.cos(11000 * np.pi * t_cont)

# ========== Muestreo INADECUADO (viola Nyquist) ==========
fs_bad = 5000  # Frecuencia de muestreo de 5 kHz (insuficiente para la componente de 5.5 kHz)
Ts_bad = 1 / fs_bad  # Periodo de muestreo = 0.0002 segundos (200 μs)
# Crea array de tiempos discretos desde 0 hasta 5ms con paso Ts_bad
n_bad = np.arange(0, 0.005, Ts_bad)
# Muestra la señal original en los tiempos discretos
x_bad = 3 * np.cos(1000 * np.pi * n_bad) + \
        5 * np.sin(2000 * np.pi * n_bad) + \
        10 * np.cos(11000 * np.pi * n_bad)

# ========== Muestreo ADECUADO (cumple Nyquist) ==========
fs_good = 12000  # Frecuencia de muestreo de 12 kHz (>2*5.5 kHz)
Ts_good = 1 / fs_good  # Periodo de muestreo ≈ 83.33 μs
# Crea array de tiempos discretos con el nuevo periodo
n_good = np.arange(0, 0.005, Ts_good)
# Muestra la señal original en los nuevos tiempos discretos
x_good = 3 * np.cos(1000 * np.pi * n_good) + \
         5 * np.sin(2000 * np.pi * n_good) + \
         10 * np.cos(11000 * np.pi * n_good)

# ========== Visualización ==========
plt.figure(figsize=(12, 6))  # Crea figura de 12x6 pulgadas

# Grafica la señal continua original (convertimos tiempo a milisegundos *1000)
plt.plot(t_cont * 1000, x_cont, label="Señal continua $x(t)$", linewidth=2)

# Superpone muestreo incorrecto (rojo) usando stem() para diagrama de líneas
plt.stem(n_bad * 1000, x_bad, linefmt='r-', markerfmt='ro', basefmt=" ",
         label="Muestreo 5kHz (con aliasing)")

# Superpone muestreo correcto (verde)
plt.stem(n_good * 1000, x_good, linefmt='g-', markerfmt='go', basefmt=" ",
         label="Muestreo 12kHz (correcto)")

# Configuración del gráfico
plt.title("Simulación de Muestreo de $x(t)$")  # Título
plt.xlabel("Tiempo [ms]")  # Etiqueta eje X (convertido a milisegundos)
plt.ylabel("Amplitud")  # Etiqueta eje Y
plt.legend()  # Muestra leyenda
plt.grid(True)  # Activa cuadrícula
plt.tight_layout()  # Ajusta espaciado
plt.show()  # Muestra el gráfico


Ejercicio 3:  En este punto buscamos hallar la distancia media entre 2 señales x1(t) y x2(t), en donde se puede hallar de la formula de la potencia media.

In [ ]:
# Ejercicio 3: Cálculo de distancia media entre señales periódicas usando SymPy
import sympy as sp  # Importa SymPy para cálculo simbólico

# ========== Definición de variables simbólicas ==========
t, T = sp.symbols('t T', real=True, positive=True)  # t: tiempo, T: periodo (ambos reales y positivos)
w0 = 2 * sp.pi / T  # Frecuencia angular fundamental (ω₀ = 2π/T)

# ========== Definición de las señales ==========
x1 = sp.cos(w0 * t)  # Señal 1: Coseno de frecuencia fundamental ω₀

# Señal 2: Onda cuadrada definida por partes (Piecewise)
x2 = sp.Piecewise(
    (1, (t >= 0) & (t < T/4)),   # Valor 1 en [0, T/4)
    (-1, (t >= T/4) & (t < 3*T/4)),  # Valor -1 en [T/4, 3T/4)
    (1, (t >= 3*T/4) & (t < T))   # Valor 1 en [3T/4, T)
)

# ========== Cálculo de la distancia media ==========
f = (x1 - x2)**2  # Función a integrar: diferencia al cuadrado entre las señales

# Integración por tramos (debido a la definición por partes de x2):
integral = (
    sp.integrate(f, (t, 0, T/4)) +  # Integral en [0, T/4)
    sp.integrate(f, (t, T/4, 3*T/4)) +  # Integral en [T/4, 3T/4)
    sp.integrate(f, (t, 3*T/4, T))  # Integral en [3T/4, T)
)

# Normalización por el periodo para obtener el valor medio
distancia_media = integral / T

# Simplificación del resultado simbólico
distancia_media_simplificada = sp.simplify(distancia_media)

# Retorna el resultado simplificado
distancia_media_simplificada

Ejercicio 4: En este punto se busca demostrar que los coeficientes de la serie exponencial de Fourier se pueden calcular utilizando la expresion matematica del problema. Ademas de hallar los coeficientes an y bn de la serie trigonometrica de Fourier con la funcion x''(t). Por ultimo tambien se pide Encuentrar el espectro de Fourier, su magnitud, fase, parte real, parte imaginaria y el error relativo de reconstruccion para n ∈{0, ±1, ±2, ±3, ±4, ±5}

In [ ]:
# Ejercicio 4

"""
Reconstrucción de una señal periódica mediante series de Fourier compleja.
Este script:
- Define una señal trapezoidal periódica.
- Calcula sus coeficientes de Fourier por dos métodos:
   1) Directamente mediante la fórmula integral.
   2) Usando los coeficientes obtenidos a partir de la segunda derivada de la señal.
- Realiza la reconstrucción aproximada de la señal usando un número finito de términos.

"""

import numpy as np
import matplotlib.pyplot as plt

# === Parámetros del sistema ===
d1 = 1    # Inicio de la rampa ascendente (izquierda)
d2 = 2    # Fin de la rampa descendente (derecha)
A = 1     # Altura máxima de la señal
T = 6     # Período total de la señal

# === Dominio temporal ===
t = np.linspace(-T/2, T/2, 1000)  # Vector de tiempo centrado en cero

# === Definición de la señal piecewise trapezoidal ===
m2 = A / (d2 - d1)  # Pendiente de las rampas
m1 = -m2            # Pendiente negativa

x = np.piecewise(t,
                  [
                      ((t >= -T/2) & (t < -d2)),        # Zona izquierda (cero)
                      ((t >= -d2) & (t <= -d1)),        # Rampa ascendente
                      ((t > -d1) & (t <= d1)),          # Plataforma superior
                      ((t > d1) & (t <= d2)),           # Rampa descendente
                      ((t > d2) & (t <= T/2))           # Zona derecha (cero)
                  ],
                  [
                      0,                                # Valor constante
                      lambda t: m1 * t + d2 * m1,       # Ecuación de la rampa ascendente
                      -A,                                # Nivel alto
                      lambda t: m2 * t + d2 * m1,       # Ecuación de la rampa descendente
                      0                                 # Valor constante
                  ])

# Arreglos para almacenar los valores reconstruidos (aceptan números complejos)
re1 = np.zeros(1000, dtype=complex)  # Reconstrucción directa
re2 = np.zeros(1000, dtype=complex)  # Reconstrucción con derivada
re3 = np.zeros(1000, dtype=complex)  # Otra reconstrucción con derivada

# === Función de reconstrucción directa ===
def recos1(ti, N, T):
    """
    Reconstruye la señal usando la serie compleja de Fourier hasta el orden N.

    Parameters:
    ti (float): Tiempo actual donde se evalúa la señal
    N (int): Número máximo de armónicos
    T (float): Período de la señal

    Returns:
    complex: Valor reconstruido en el instante ti
    """
    suma = 0
    omega = (2 * np.pi) / T  # Frecuencia angular fundamental
    for i in range(N + 1):
        if i != 0:
            # Coeficiente Cn multiplicado por exponencial compleja
            integrando = x * np.exp(-1j * i * omega * t)
            cn = (2 / T) * np.trapezoid(integrando, t)
            suma += cn * np.exp(1j * i * omega * ti)
        else:
            # Término constante (C0)
            suma += (1 / T) * np.trapezoid(x, t)
    return suma

# === Función de reconstrucción usando la segunda derivada ===
def recos2(ti, N, T, d1, d2, A):
    """
    Reconstruye la señal usando coeficientes calculados a partir de la segunda derivada.

    Parameters:
    ti (float): Tiempo actual donde se evalúa la señal
    N (int): Número máximo de armónicos
    T (float): Período de la señal
    d1, d2 (float): Límites de las rampas
    A (float): Altura máxima de la señal

    Returns:
    complex: Valor reconstruido en el instante ti
    """
    suma = 0
    omega = (2 * np.pi) / T
    for i in range(N + 1):
        if i != 0:
            # Coeficiente Cn usando derivada
            numerador = 4 * (A / (d2 - d1)) * (np.cos(d1 * i * omega) - np.cos(d2 * i * omega))
            denominador = T * i**2 * omega**2
            cn = numerador / denominador
            suma += cn * np.exp(1j * i * omega * ti)
        else:
            # Término constante (C0), igual que antes
            suma += (1 / T) * np.trapezoid(x, t)
    return suma

# === Reconstrucción punto a punto ===
for i in range(1000):
    re1[i] = recos1(t[i], 4, T)  # Reconstrucción con 5 términos (0 a 4)
for i in range(1000):
    re2[i] = recos2(t[i], 5, T, d1, d2, -A)  # Reconstrucción con 6 términos (0 a 5)
# for i in range(1000):
#     re3[i] = recos2(t[i], 1, T, d1, d2, -A)  # Reconstrucción con 2 términos (0 a 1)

# === Gráfica de resultados ===
plt.figure(figsize=(10, 6))
plt.plot(t, x, color=(0.0588, 0.1843, 0.9804), label='Señal original')
plt.plot(t, re1.real, color=(0.2549, 0.9804, 0.4235), label='Reconstrucción directa (N=4)')
plt.plot(t, re2.real, color=(0.9804, 0.1059, 0.1882), label='Reconstrucción derivada (N=5)')
#plt.plot(t, re3.real, color=(0,0,0), label='Reconstrucción derivada (N=1)')
plt.title('Reconstrucciones de la señal usando Serie Compleja de Fourier')
plt.xlabel('Tiempo (t)')
plt.ylabel('Amplitud')
plt.legend()
plt.grid(color=(0.0157, 0.0431, 0.2314))
plt.tight_layout()
plt.show()

In [ ]:
#@title Cálculo del espectro:

# === Calcular término constante c₀ ===
c0 = (1 / T) * np.trapezoid(x, t)

N_max = 5   # Número máximo de armónicos a graficar
# === Vector de armónicos n ∈ [-N_max, ..., 0, ..., N_max] ===
nv = np.arange(-N_max, N_max + 1)

# === Frecuencia angular fundamental ===
omega0 = (2 * np.pi) / T

# === Calcular coeficientes de Fourier usando segunda derivada ===
Cn = np.zeros_like(nv, dtype=np.complex128)

non_zero_mask = nv != 0  # Evitar división por cero
Cn[non_zero_mask] = (2j / (T * nv[non_zero_mask]**2 * omega0**2)) * \
                    (np.sin(nv[non_zero_mask] * omega0 * d1) - np.sin(nv[non_zero_mask] * omega0 * d2))

# Asignar valor medio en n=0
Cn[nv == 0] = c0

# Gráfica del espectro
plt.figure(figsize=(7, 3))
plt.stem(nv, np.abs(Cn))
plt.title("Espectro de Magnitud |cₙ| usando segunda derivada")
plt.xlabel("n (armónico)")
plt.ylabel("|cₙ|")
plt.grid(True)
plt.axhline(0, color='black', linewidth=0.8)
plt.xticks(nv)
plt.tight_layout()
plt.show()

grafica del espectro, con su parte real e imaginaria.

In [ ]:
# @title graficar espectro

# Gráfico del espectro en formato de matriz 2x2
fig = plt.figure(figsize=(7, 6))

# === Fila 1: Parte real e imaginaria ===

# Parte real
ax1 = fig.add_subplot(2, 2, 1)
ax1.stem(nv, np.real(Cn), 'r', basefmt=" ")
ax1.set_title(r'Parte Real $\text{Re}\{C_n\}$', fontsize=14)
ax1.set_xlabel(r'$n$', fontsize=12)
ax1.set_ylabel(r'$\text{Re}\{C_n\}$', fontsize=12)
ax1.grid(True)
ax1.axhline(0, color='black', lw=0.8)
ax1.set_xticks(nv)

# Parte imaginaria
ax2 = fig.add_subplot(2, 2, 2)
ax2.stem(nv, np.imag(Cn), 'r', basefmt=" ")
ax2.set_title(r'Parte Imaginaria $\text{Im}\{C_n\}$', fontsize=14)
ax2.set_xlabel(r'$n$', fontsize=12)
ax2.set_ylabel(r'$\text{Im}\{C_n\}$', fontsize=12)
ax2.grid(True)
ax2.axhline(0, color='black', lw=0.8)
ax2.set_xticks(nv)

# Ajustar espaciado entre subplots
fig.tight_layout()
plt.show()